In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Possible business questions:

- What factors contribute most heavily to the price? Make a model to predict the price - one that's interpretable and perhaps one (like a neural network) that's not interpretable but does a better job of price prediction. Which listings are underpriced - meaning that the host could make more money by increasing the price, or travelers can get a good deal by booking them?
- What should those listings with poor overall review scores do in order to get better reviews in the future?
- What can we discern about the character of each neighborhood using the descriptions?

In [2]:
listings = pd.read_csv("listings.csv")
calendar = pd.read_csv("calendar.csv")

Definitions for many (though not all) of the fields can be found [here](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896).

Drop columns that have all `NA` values.

In [3]:
for df in [listings, calendar]:
    df.dropna(axis=1, how='all', inplace=True)

Drop rows in `listings` that have `NA` in the `price` field, and do the same for the `price` and `adjusted_price` fields in `calendar`.

In [4]:
listings.dropna(subset=['price'], inplace=True)
calendar.dropna(subset=['price', 'adjusted_price'], inplace=True)

Observe that a dollar sign preceeds the number in several columns; I will remove this so that the field is treated as numeric.

In [5]:
listings['price'] = listings.apply((lambda x: x['price'].replace('$','').replace(',','')), axis=1)
calendar['price'] = calendar.apply((lambda x: x['price'].replace('$','').replace(',','')), axis=1)
calendar['adjusted_price'] = calendar.apply((lambda x: x['adjusted_price'].replace('$','').replace(',','')), axis=1)

In [6]:
listings['price'] = listings['price'].astype(np.float64)
calendar['price'] = calendar['price'].astype(np.float64)
calendar['adjusted_price'] = calendar['adjusted_price'].astype(np.float64)

Similarly, some of the fields are expressed as percentages; I will remove the percent signs and change the entries to floats.

In [7]:
listings['host_response_rate'] = listings.apply(lambda x: float(x['host_response_rate'].replace('%','')) if pd.notnull(x['host_response_rate']) else x['host_response_rate'], axis=1)
listings['host_acceptance_rate'] = listings.apply(lambda x: float(x['host_acceptance_rate'].replace('%','')) if pd.notnull(x['host_acceptance_rate']) else x['host_acceptance_rate'], axis=1)

## 1. Price prediction

Split into `X` and `y`.

In [8]:
X = listings.drop('price', axis=1)
y = listings['price']

I also need to deal with dates appropriately. The relevant columns are `host_since`, `first_review`, and `last_review`. I could scrap these columns altogether, but it might be useful to know the length of time since the date for each row. 

In [9]:
for col in ['host_since', 'first_review', 'last_review']:
    X[col] = pd.to_datetime(X['last_scraped']) - pd.to_datetime(X[col])
    X[col] = X.apply(lambda x: x[col].days, axis=1)

Certain columns in `X` are not useful for prediction (for example, URLs), and should be removed.

In [10]:
to_drop = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'name']
to_drop += ['description', 'neighborhood_overview', 'picture_url', 'host_id']
to_drop += ['host_url', 'host_name', 'host_about', 'host_thumbnail_url']
to_drop += ['host_picture_url', 'license', 'calendar_last_scraped']
X.drop(labels=to_drop, axis=1, inplace=True)

Certain columns (`host_verifications` and `amenities`) are lists of options; I'd like to split these into different columns.

The first step is to change these entries into actual Python lists instead of strings (annoyingly, the two relevant columns are formatted slightly differently, with the roles of single and double quotes being interchanged between them).

In [11]:
X['host_verifications'] = X.apply(lambda x: x['host_verifications'].replace('[','').replace(']','').replace('\'','').split(', '), axis=1)

In [12]:
X['amenities'] = X.apply(lambda x: x['amenities'].replace('\"','').replace('[','').replace(']','').split(', '), axis=1)

What are the possible values in the `host_verifications` lists?

In [13]:
distinct_host_verifications = set()
def find_distinct_hvs(x):
    for hv in x['host_verifications']:
        distinct_host_verifications.add(hv)
X.apply(lambda x: find_distinct_hvs(x), axis=1);

In [14]:
len(distinct_host_verifications)

20

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer

In [16]:
m = MultiLabelBinarizer()
hv_df = pd.DataFrame(m.fit_transform(X['host_verifications']), columns=m.classes_, index=X.index)

In [17]:
X = pd.concat([X, hv_df], axis=1)

In [18]:
X.rename(columns={'None' : 'HV_None'}, inplace=True)
# I don't want a column to be called just 'None', so I changed it a bit

Now I can drop the original `host_verifications` column.

In [19]:
X.drop('host_verifications', axis=1, inplace=True)

Now I'd like to do the same for the `amenities` column. What are the possible values in those lists?

In [20]:
distinct_amenities = set()
def find_distinct_amenities(x):
    for a in x['amenities']:
        distinct_amenities.add(a)
X.apply(lambda x: find_distinct_amenities(x), axis=1)
len(distinct_amenities)

990

Ok, that's a lot of amenities. I don't really want to vastly blow up the number of features like this, so I'll just take the 20 most popular amenities.

In [21]:
from collections import defaultdict
from operator import itemgetter

In [22]:
amenity_counts = defaultdict(int) # Initializes to zero for each value
def update_counts(x):
    for a in x['amenities']:
        amenity_counts[a] += 1
X.apply(lambda x: update_counts(x), axis=1);

In [23]:
top_20 = sorted(amenity_counts.items(), key=itemgetter(1), reverse=True)[:20]

In [24]:
top_20[:5]

[('Smoke alarm', 6359),
 ('Essentials', 6185),
 ('Wifi', 6141),
 ('Kitchen', 5930),
 ('Carbon monoxide alarm', 5926)]

In [25]:
top_20_amenities = dict(top_20).keys()

Now I want to get rid of the other amenities in each row.

In [26]:
def find_amenities(x):
    amenities = []
    for a in top_20_amenities:
        if a in x['amenities']:
            amenities.append(a)
    return amenities

In [27]:
X['amenities'] = X.apply(lambda x: find_amenities(x), axis=1)

Now I can create the dummy variables and drop the original `amenities` column.

In [28]:
m = MultiLabelBinarizer()
amenities_df = pd.DataFrame(m.fit_transform(X['amenities']), columns=m.classes_, index=X.index)
X = pd.concat([X, amenities_df], axis=1)
X.drop('amenities', axis=1, inplace=True)

Next, I'll get dummies for the rest of the categorical variables.

In [29]:
X = pd.get_dummies(X)

In [30]:
X.shape

(6528, 753)

Now I need to deal with `NA` values for the numeric columns. There's no totally ideal way to do this, but to move forward I'll impute using the mean.

In [31]:
fill_mean = lambda col: col.fillna(col.mean())
X = X.apply(fill_mean)